In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Conv1D, GlobalMaxPooling1D, Activation, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [2]:
# Set GPU device
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print('GPU device set')
    except RuntimeError as e:
        print(e)

GPU device set


In [3]:
# Load the data
train_data = pd.read_csv('/kaggle/input/swm-data-final-split/swm_data_final_split/new_data_final_train.csv')
val_data = pd.read_csv('/kaggle/input/swm-data-final-split/swm_data_final_split/new_data_final_val.csv')
test_data = pd.read_csv('/kaggle/input/swm-data-final-split/swm_data_final_split/new_data_final_test.csv')

**Preprocessing**

In [4]:
def assign_class(label):
  if label == "OR":
    return 0
  else:
    return 1

In [5]:
train_data['label_final'] = train_data['label'].apply(assign_class)
val_data['label_final'] = val_data['label'].apply(assign_class)
test_data['label_final'] = test_data['label'].apply(assign_class)

In [6]:
# Separate out text and labels for each dataset
train_text = train_data["text_final"].values.astype("str")
train_labels = train_data["label_final"].values
val_text = val_data["text_final"].values.astype("str")
val_labels = val_data["label_final"].values
test_text = test_data["text_final"].values.astype("str")
test_labels = test_data["label_final"].values

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
def generate_inputs(texts, tokenizer, max_length):
    # Tokenize the input texts
    tokens = tokenizer.batch_encode_plus(
        texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

    # Extract the token ids, attention masks, and token type ids
    input_ids = np.array(tokens['input_ids'])
    attention_masks = np.array(tokens['attention_mask'])
    token_type_ids = np.array(tokens['token_type_ids'])
        
    # Return the inputs as a list of NumPy arrays
    return [input_ids, attention_masks, token_type_ids]

In [9]:
max_length = 128

In [10]:
# Tokenize the train, validation, and test texts
train_tokens = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens = True,
    max_length=max_length,
    padding = True,
    return_attention_mask = True,
    truncation=True,
    return_tensors='tf'
)
val_tokens = tokenizer.batch_encode_plus(
    val_text,
    add_special_tokens = True,
    max_length=max_length,
    padding = True,
    return_attention_mask = True,
    truncation=True,
    return_tensors='tf'
)
test_tokens = tokenizer.batch_encode_plus(
    test_text,
    add_special_tokens = True,
    max_length=max_length,
    padding = True,
    return_attention_mask = True,
    truncation=True,
    return_tensors='tf'
)

# Extract the token ids, attention masks, and token type ids for train, validation, and test
train_input_ids = np.array(train_tokens['input_ids'])
train_attention_masks = np.array(train_tokens['attention_mask'])
train_token_type_ids = np.array(train_tokens['token_type_ids'])
val_input_ids = np.array(val_tokens['input_ids'])
val_attention_masks = np.array(val_tokens['attention_mask'])
val_token_type_ids = np.array(val_tokens['token_type_ids'])
test_input_ids = np.array(test_tokens['input_ids'])
test_attention_masks = np.array(test_tokens['attention_mask'])
test_token_type_ids = np.array(test_tokens['token_type_ids'])

In [11]:
# Generate the input data for train, validation, and test
train_inputs = generate_inputs(train_text, tokenizer, max_length)
val_inputs = generate_inputs(val_text, tokenizer, max_length)
test_inputs = generate_inputs(test_text, tokenizer, max_length)

# Generate the target data for train, validation, and test
train_targets = np.array(train_labels)
val_targets = np.array(val_labels)
test_targets = np.array(test_labels)

**Model**

In [12]:
# Load BERT model
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=False)

In [13]:
# Build model
input_word_ids = Input(shape=(None,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(None,), dtype=tf.int32, name="input_mask")
input_type_ids = Input(shape=(None,), dtype=tf.int32, name="input_type_ids")
bert_inputs = {"input_word_ids": input_word_ids, "input_mask": input_mask, "input_type_ids": input_type_ids}

# BERT embeddings
bert_outputs = bert_layer(bert_inputs)
pooled_output = bert_outputs["pooled_output"]
sequence_output = bert_outputs["sequence_output"]

# CNN layer
conv_layer_1 = Conv1D(filters=64, kernel_size=2, padding="same", activation="relu")(sequence_output)
conv_layer_2 = Conv1D(filters=64, kernel_size=3, padding="same", activation="relu")(conv_layer_1)
max_pooling_layer = GlobalMaxPooling1D()(conv_layer_2)

# Attention layer
att_output = Attention()([max_pooling_layer, max_pooling_layer])

# Classification layer
dropout_layer = Dropout(0.2)(att_output)
output_layer = Dense(1, activation="sigmoid")(dropout_layer)

In [14]:
# Define model inputs and outputs
model = Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output_layer)

# Compile model
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              metrics=["accuracy"])

# Set up early stopping and model checkpoint callbacks
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5),
    ModelCheckpoint(filepath="best_model.h5", monitor="val_loss", save_best_only=True,
                   save_weights_only=True, verbose=1)
]

In [15]:
# Define the training parameters
batch_size = 64
epochs = 20

# Train the model
history = model.fit(train_inputs, train_targets, validation_data=(val_inputs, val_targets), 
                    epochs=epochs, batch_size=batch_size, callbacks=callbacks)

Epoch 1/20
506/506 [==============================] - ETA: 0s - loss: 0.6968 - accuracy: 0.5005
Epoch 1: val_loss improved from inf to 0.69304, saving model to best_model.h5
506/506 [==============================] - 189s 342ms/step - loss: 0.6968 - accuracy: 0.5005 - val_loss: 0.6930 - val_accuracy: 0.5080
Epoch 2/20
506/506 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.4997
Epoch 2: val_loss did not improve from 0.69304
506/506 [==============================] - 171s 338ms/step - loss: 0.6933 - accuracy: 0.4997 - val_loss: 0.6933 - val_accuracy: 0.4920
Epoch 3/20
506/506 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5029
Epoch 3: val_loss did not improve from 0.69304
506/506 [==============================] - 171s 338ms/step - loss: 0.6932 - accuracy: 0.5029 - val_loss: 0.6934 - val_accuracy: 0.4922
Epoch 4/20
506/506 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5009
Epoch 4: val_loss did not improve from 

In [16]:
def evaluate_model(model, X_test, y_test):
    y_pred = np.round(model.predict(X_test))
    y_true = y_test

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict(X_test), multi_class='ovr')

    metrics = {'accuracy': accuracy,
               'precision': precision,
               'recall': recall,
               'f1': f1,
               'confusion_matrix': cm,
               'roc_auc': roc_auc}

    return metrics

In [17]:
# Load the best saved model
model.load_weights("/kaggle/working/best_model.h5")

# Evaluate the model on the test data
test_metrics = evaluate_model(model, test_inputs, test_targets)

# Print the evaluation metrics
print('Test accuracy:', test_metrics['accuracy'])
print('Test precision:', test_metrics['precision'])
print('Test recall:', test_metrics['recall'])
print('Test F1 score:', test_metrics['f1'])
print('Test ROC-AUC score:', test_metrics['roc_auc'])
print('Test confusion matrix:\n', test_metrics['confusion_matrix'])

127/127 [==============================] - 19s 154ms/step
Test accuracy: 0.51162215628091
Test precision: 0.23076923076923078
Test recall: 0.001524390243902439
Test F1 score: 0.0030287733467945486
Test ROC-AUC score: 0.5024551053855915
Test confusion matrix:
 [[2066   10]
 [1965    3]]
